In [8]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.svm import LinearSVC

파싱

In [9]:
def parsing(path):#파싱을 진행하는 함수
    with open(path,'r',encoding='utf-8') as f:
        #파일을 읽어들이고 ['로그','로그',...] 이런식으로 로그를 구조화
        train = []
        log_data = "" # 하나의 로그에 대해 추출한 url path와 query, payload 값을 " "로 구분한 긴 문자열
        para="" # 하나의 로그에 대해 모든 내용을 담고 있는 문자열
        while True:
            l = f.readline() #한줄씩 읽어 옵니다
            if not l:
                break #파일을 전부 읽으면 읽기를 중단합니다.

            if l != "\n":
                para +=l
                if l[:4] == 'GET ' or l[:4] == 'PUT ':
                    # GET method나 PUT method일 경우 url의 path와 query 추출
                    url = l[4:len(l)-10]
                    # url 은 method type과 HTTP/1.1을 뺀 중간 부분
                    url_path = url[21:]
                    # url에서 domain인 http://localhost:8080 제거 
                    query_start = url_path.find('?')
                    if (query_start != -1):
                        # query 문이 있을 경우 path와 query들을 분리
                        log_data += url_path[:query_start] + " "
                        log_data += " ".join(url_path[query_start+1:len(url)].split("&")) + " "
                    else:
                        # query 문이 없을 경우 path 만 저장
                        log_data += url_path + " "   

                elif l[:5] == 'POST ':
                    # POST method일 경우 url의 path 추출
                    url = l[5:len(l)-10]
                    url_path = url[21:]
                    log_data += url_path + " "
                                          
            else:
                if para!='':
                    if para[:4]=='POST' or para[:3] == 'PUT': #Method가 POST나 PUT일 경우 예외적으로 바디까지 가져옵니다.
                        log_data += " ".join(f.readline().strip().split("&")) + " "
                    # print(log_data)
                    train.append(log_data)
                    para=""
                    log_data =""
                    # print("=============================================")
    return train

In [10]:
def dataset(path,mod='train'): #데이터셋을 생성합니다. 파싱한 데이터와 라벨을 생성합니다 
    x = parsing(f'{path}norm_{mod}.txt') # mod에 따라 train을 가져올지 test 데이터를 가져올지 결정됩니다.
    y = [0]*len(x)
    x += parsing(f'{path}anomal_{mod}.txt')
    y += [1]*(len(x)-len(y))
    return x, y

In [11]:
def vectorize(train_x,test_x): #문장을 벡터로 만듭니다 해당 코드에서는 기본적인 tf idf를 사용하고 있습니다.
    tf = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3,3))
    tf = tf.fit(train_x)
    train_vec = tf.transform(train_x)
    test_vec = tf.transform(test_x)
    return train_vec,test_vec

In [12]:
def train(train_vec,train_y): #Linear svm으로 훈련 시킵니다.
    linear_svm = LinearSVC()
    linear_svm.fit(train_vec, train_y)
    return linear_svm

In [13]:
def test(test_y,test_vec, svm): #입렵 받은 테스트와 모델로 테스트를 실시합니다
    pred = svm.predict(test_vec)
    print(accuracy_score(test_y,pred))
    print(f1_score(test_y,pred))
    return pred

In [14]:
############### 실행 코드 #######################
################################################
train_x, train_y = dataset('./','train')
test_x, test_y =  dataset('./','test')
train_vec, test_vec = vectorize(train_x, test_x)
svm = train(train_vec, train_y)
pred = test(test_y,test_vec, svm)

# ########################################################
# tf = TfidfVectorizer()
# tf = tf.fit(train_x)

# print(len(tf.vocabulary_)) # 고유한 단어가 대략 3만개가 나옵니다

# print(tf.transform(train_x)[0])

0.9922214034225825
0.9904800080168353
